<a href="https://colab.research.google.com/github/PatriciaLucas/CausalForecasting/blob/main/GrafoCausal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geração de grafo causal para séries temporais multivariadas

```
Universidade Federal de Minas Gerais - UFMG
Disciplina: Causalidade em Aprendizado de Máquina (DCC)
Projeto Final: Modelo Causal para Previsão de Séries Temporais Multivariadas
Professor: Anísio Mendes Lacerda
```

###Instalação e importação de pacotes

In [ ]:
!pip install -qU git+https://github.com/stbachinger/TigramiteGui.git

In [ ]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb
from tigramite.models import LinearMediation, Prediction

/usr/local/lib/python3.7/dist-packages/tigramite/independence_tests/gpdc.py:16: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/usr/local/lib/python3.7/dist-packages/tigramite/independence_tests/gpdc_torch.py:18: UserWarning: No module named 'dcor'
  warnings.warn(str(e))


###Base de dados:

In [ ]:
jequitinhonha = pd.read_csv("https://raw.githubusercontent.com/PatriciaLucas/CausalForecasting/main/Datasets/jequitinhonha.csv")
mucuri = pd.read_csv("https://raw.githubusercontent.com/PatriciaLucas/CausalForecasting/main/Datasets/mucuri.csv")
riodoce = pd.read_csv("https://raw.githubusercontent.com/PatriciaLucas/CausalForecasting/main/Datasets/riodoce.csv")

###Verifica estacionariedade nas séries:

In [ ]:
def fuller(data):
  from statsmodels.tsa.stattools import adfuller
  X = data.values
  result = adfuller(X) 
  if result[1] > 0.05:
    return False
  else:
    return True

In [ ]:
for i in range(1,9):
  print("Variável: ", i, "Estacionária: ", fuller(riodoce.iloc[:,i]))

Variável:  1 Estacionária:  True
Variável:  2 Estacionária:  True
Variável:  3 Estacionária:  True
Variável:  4 Estacionária:  True
Variável:  5 Estacionária:  True
Variável:  6 Estacionária:  True
Variável:  7 Estacionária:  True
Variável:  8 Estacionária:  True


## Geração do grafo causal com o método PCMCI

###Variável $ET_o$:

In [ ]:
data = riodoce
dataset = np.empty((11505,9))
dataset = np.concatenate((data['T_MAX'].values.reshape((-1,1)), data['T_MIN'].values.reshape((-1,1)), data['UH'].values.reshape((-1,1)), 
                         data['V'].values.reshape((-1,1)), data['R'].values.reshape((-1,1)), data['ETO'].values.reshape((-1,1))), axis=1)
var_names = [r'Tmax', r'Tmin', r'UR', r'V', r'R', r'Eto']
dataframe = pp.DataFrame(dataset, datatime = {0:np.arange(len(data))}, var_names=var_names)

In [ ]:
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
results = pcmci.run_pcmci(tau_max=5, pc_alpha=0.01, alpha_level=0.01)

In [ ]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=5, fdr_method='fdr_bh')
pcmci.print_significant_links(
        p_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.01)
graph = pcmci.get_graph_from_pmatrix(p_matrix=q_matrix, alpha_level=0.01, 
            tau_min=0, tau_max=5, selected_links=None)
results['graph'] = graph


## Significant links at alpha = 0.01:

    Variable Tmax has 17 link(s):
        (Eto  0): pval = 0.00000 | val =  0.667
        (UR  0): pval = 0.00000 | val = -0.658
        (R  0): pval = 0.00000 | val =  0.566
        (Tmax -1): pval = 0.00000 | val =  0.367
        (V  0): pval = 0.00000 | val = -0.192
        (Tmin  0): pval = 0.00000 | val =  0.147
        (V -2): pval = 0.00000 | val =  0.098
        (Eto -1): pval = 0.00000 | val =  0.085
        (V -1): pval = 0.00000 | val = -0.083
        (UR -1): pval = 0.00000 | val = -0.081
        (Tmin -1): pval = 0.00000 | val =  0.081
        (UR -2): pval = 0.00000 | val =  0.048
        (Tmax -4): pval = 0.00003 | val =  0.041
        (R -2): pval = 0.00007 | val = -0.040
        (Tmax -2): pval = 0.00007 | val = -0.040
        (Eto -2): pval = 0.00008 | val = -0.039
        (UR -3): pval = 0.00798 | val = -0.027

    Variable Tmin has 26 link(s):
        (Tmin -1): pval = 0.00000 | val =  0.484
        (Tmax -1): pval = 0.00000 |

###Variável $T_{MAX}$, $T_{MIN}$, $UR$, $V$, $R$:

In [ ]:
data = riodoce
dataset = np.empty((11505,9))
dataset = np.concatenate((data['T_MAX'].values.reshape((-1,1)), data['T_MIN'].values.reshape((-1,1)), data['UH'].values.reshape((-1,1)), 
                         data['V'].values.reshape((-1,1)), data['R'].values.reshape((-1,1))), axis=1)
var_names = [r'Tmax', r'Tmin', r'UR', r'V', r'R']
dataframe = pp.DataFrame(dataset, datatime = {0:np.arange(len(data))}, var_names=var_names)

parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
results = pcmci.run_pcmci(tau_max=5, pc_alpha=0.01, alpha_level=0.01)

q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=5, fdr_method='fdr_bh')
pcmci.print_significant_links(
        p_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.01)
graph = pcmci.get_graph_from_pmatrix(p_matrix=q_matrix, alpha_level=0.01, 
            tau_min=0, tau_max=5, selected_links=None)
results['graph'] = graph


## Significant links at alpha = 0.01:

    Variable Tmax has 14 link(s):
        (UR  0): pval = 0.00000 | val = -0.657
        (R  0): pval = 0.00000 | val =  0.567
        (Tmax -1): pval = 0.00000 | val =  0.397
        (V  0): pval = 0.00000 | val = -0.191
        (Tmin  0): pval = 0.00000 | val =  0.150
        (V -2): pval = 0.00000 | val =  0.101
        (UR -1): pval = 0.00000 | val = -0.091
        (Tmin -1): pval = 0.00000 | val =  0.090
        (R -1): pval = 0.00000 | val =  0.086
        (V -1): pval = 0.00000 | val = -0.058
        (UR -2): pval = 0.00000 | val =  0.046
        (Tmax -2): pval = 0.00086 | val = -0.034
        (R -2): pval = 0.00151 | val = -0.033
        (Tmax -4): pval = 0.00223 | val =  0.031

    Variable Tmin has 20 link(s):
        (Tmin -1): pval = 0.00000 | val =  0.484
        (Tmax -1): pval = 0.00000 | val =  0.475
        (UR -1): pval = 0.00000 | val =  0.350
        (UR  0): pval = 0.00000 | val =  0.234
        (V -1): pval = 0.00000 | val 